# 0. init

In [ ]:
# WARNING: Comment if code doesn't run
# Use jit to compile and optimize Python code
from numba import jit

# Arrays and analysis
import numpy as np
import scipy as sp
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit

# Plotting and config
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['figure.constrained_layout.use'] = True

# Network
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

# Misc imports
from slugify import slugify
from functools import partial
import os

# Important directories
FIG_DIR = 'fig/'
DATA_DIR = 'data/'
DUMP_DIR = 'dump/'

def save_fig(title):
    """Save figure under normalized name."""
    plt.savefig(f'{FIG_DIR}/{slugify(title)}.png', bbox_inches='tight')

def props_fig(props_dict):
    """Update current Axes with the given dictionary of properties."""
    plt.gca().update(props_dict)

def create_dirs(path):
    """Create directory, do nothing if it exists."""
    os.makedirs(path, exist_ok=True)

create_dirs(FIG_DIR)
create_dirs(DATA_DIR)
create_dirs(DUMP_DIR)

# Networks

### 2.1 Implement SIR Disease Spread on Network 

In [ ]:
def SIR_network(graph, beta, gamma, fraction_infected, model_it):
    ''' Run the SIR model on a network '''
    
    # Model selection
    model = ep.SIRModel(graph)
    
    # Model Configuration
    cfg = mc.Configuration()
    cfg.add_model_parameter('beta', beta)
    cfg.add_model_parameter('gamma', gamma)
    cfg.add_model_parameter("fraction_infected", fraction_infected)
    model.set_initial_status(cfg)
    
    # Simulation execution
    iterations = model.iteration_bunch(model_it)
    
    # Extract S, I, R trends
    Susceptible = [iteration['node_count'][0] for iteration in iterations]
    Infected = [iteration['node_count'][1] for iteration in iterations]
    Recovered = [iteration['node_count'][2] for iteration in iterations]

    return Susceptible, Infected, Recovered

    
def viz_subplot(ax, time, all_susceptible, all_infected, all_recovered, avg_susceptible, avg_infected, avg_recovered, title):
    ''' Plot different trajectories for Susceptible, Infected and Recoveren
    and the average trajectorie '''

    for susceptible, infected, recovered in zip(all_susceptible, all_infected, all_recovered):
        ax.plot(time, susceptible, alpha=.2, color="red")
        ax.plot(time, infected, alpha=.2, color="green")
        ax.plot(time, recovered, alpha=.2, color="blue")
    
    # Plot the averages
    ax.plot(time, avg_susceptible, color="red", label="S (average)", linewidth=2)
    ax.plot(time, avg_infected, color="green", label="I (average)", linewidth=2)
    ax.plot(time, avg_recovered, color="blue", label="R (average)", linewidth=2)

    ax.set_title(title)
    

def SIR_network_plot(time, network, betas, gammas, fraction_infected, model_it, n_it):
    ''' Compute the data for different SIR trajectories on a network for different parameter values,
    combining the subplots one figure '''

    dbeta = len(betas)
    dgamma = len(gammas)
    fig, axs = plt.subplots(dbeta, dgamma, figsize=(5*dbeta, 4*dgamma))
    axs = axs.flatten()
     
    j=0
    for beta in betas:
        for gamma in gammas:
            n_time_points = len(time)  

            # Create arrays
            all_S = np.zeros((n_it, n_time_points))
            all_I = np.zeros((n_it, n_time_points))
            all_R = np.zeros((n_it, n_time_points))

            for i in range(n_it):
                S, I, R = SIR_network(network, beta, gamma, fraction_infected, model_it)
                
                all_S[i]=S
                all_I[i]=I
                all_R[i]=R
            
            # Compute averages over all iterations
            avg_S = np.mean(all_S, axis=0)
            avg_I = np.mean(all_I, axis=0)
            avg_R = np.mean(all_R, axis=0)
                
            title = f"Beta: {beta}, Gamma: {gamma}"
            viz_subplot(axs[j], time, all_S, all_I, all_R, avg_S, avg_I, avg_R, title)
            j+=1
    fig.suptitle(network_name(network), fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 1])
    props_fig()
    save_fig(network_name(network))
    plt.show()



### 2.2 Generating Networks

In [ ]:
# Network name
def network_name(network):
    ''' Function to assign names to the networks'''
    if network == g_ER:
        network_name = "Erdos-Reyni Network"
    elif network == g_BA:
        network_name = "Barbasi-Albert Network"
    elif network == g_WS:
        network_name = "Watts-Strogatz Network"
    return network_name

### 2.3 Network Statistics

In [ ]:

def statistics_plot(ax, network, statistics):
    ''' Plots the given statistics in a frequency plot'''
    
    title = (f"Frequency of {statistic_name(statistics)} in a {network_name(network)}")

    ax.hist(statistics, bins=100, color='blue')
    #ax.tight_layout()
    ax.set_xlabel("value")
    ax.set_ylabel("count")
    ax.set_title(title)
    #plt.show()
    #save_fig(title)
    
def statistics_viz(ax, network, statistics):
    ''' Visualizes the given statistics in a graph'''

    title = (f"Visualization of {statistic_name(statistics)} in a small {network_name(network)}")

    # Generate layout for the graph
    format = nx.spring_layout(network)

    # Set nodes colors with color mapping based on given statistics
    node_colors = np.array(statistics) 

    # Draw nodes and edges
    nodes = nx.draw_networkx_nodes(network, format, cmap=plt.cm.plasma, node_size = 50 ,node_color = node_colors,  edgecolors = 'black', linewidths= 0.5, ax=ax)
    nx.draw_networkx_edges(network, format , edge_color= 'black', alpha =0.4, ax=ax)

    # Add a color bar to the plot
    plt.colorbar(nodes, ax=ax)

    ax.set_title(title)
    ax.set_axis_off()


def statistic_name(statistics):
    if statistics == centrality_values:
        name = "Degree centrality"
    if statistics == betweenness_values:
        name = "Betweenness centrality"
    if statistics == clustering_values:
        name = "Clustering coefficient"
    return name

### 2.3.1 Erdos-Reyni Graph

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16, 10))
axs = axs.flatten()
it = 100

all_cent = np.zeros((it,1000))
all_betw = np.zeros((it,1000))
all_clus = np.zeros((it,1000))

for t in range(it):
    # Network topology: Large graph for statistics
    g_ER = nx.erdos_renyi_graph(1000, 0.1)          
    network = g_ER

    # Centrality
    centrality = nx.degree_centrality(network)
    centrality_values = list(centrality.values())

    # Betweenness
    betweenness = nx.betweenness_centrality(network)
    betweenness_values = list(betweenness.values())

    # clustering
    clustering = nx.clustering(network)
    clustering_values = list(clustering.values())

    all_cent[t] = centrality_values
    all_betw[t] = betweenness_values
    all_clus[t] = clustering_values

centrality_values = [sum(group) / len(group) for group in zip(*all_cent)]
betweenness_values = [sum(group) / len(group) for group in zip(*all_betw)]
clustering_values = [sum(group) / len(group) for group in zip(*all_clus)]

statistics_plot(axs[0], network, centrality_values)
statistics_plot(axs[2], network, betweenness_values)
statistics_plot(axs[4], network, clustering_values)


# Network topology: small graph for visualization
g_ER = nx.erdos_renyi_graph(150, 0.1)  #Erdos-Reyni graph
network = g_ER

# Centrality
centrality = nx.degree_centrality(network)
centrality_values = list(centrality.values())

# Betweenness
betweenness = nx.betweenness_centrality(network)
betweenness_values = list(betweenness.values())

# clustering
clustering = nx.clustering(network)
clustering_values = list(clustering.values())

statistics_viz(axs[1], network, centrality_values)
statistics_viz(axs[3], network, betweenness_values)
statistics_viz(axs[5], network, clustering_values)


plt.show

### 2.3.2 Barabasi-Albert Graph

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16, 10))
axs = axs.flatten()

it = 100

all_cent = np.zeros((it,1000))
all_betw = np.zeros((it,1000))
all_clus = np.zeros((it,1000))

for t in range(it):
    # Network topology: Large graph for statistics
    g_BA = nx.barabasi_albert_graph(1000, 6)          
    network = g_BA

    # Centrality
    centrality = nx.degree_centrality(network)
    centrality_values = list(centrality.values())

    # Betweenness
    betweenness = nx.betweenness_centrality(network)
    betweenness_values = list(betweenness.values())

    # clustering
    clustering = nx.clustering(network)
    clustering_values = list(clustering.values())

    all_cent[t] = centrality_values
    all_betw[t] = betweenness_values
    all_clus[t] = clustering_values

centrality_values = [sum(group) / len(group) for group in zip(*all_cent)]
betweenness_values = [sum(group) / len(group) for group in zip(*all_betw)]
clustering_values = [sum(group) / len(group) for group in zip(*all_clus)]

statistics_plot(axs[0], network, centrality_values)
statistics_plot(axs[2], network, betweenness_values)
statistics_plot(axs[4], network, clustering_values)


# Network topology: small graph for visualization
g_BA = nx.barabasi_albert_graph(150, 6)          
network = g_BA

# Centrality
centrality = nx.degree_centrality(network)
centrality_values = list(centrality.values())

# Betweenness
betweenness = nx.betweenness_centrality(network)
betweenness_values = list(betweenness.values())

# clustering
clustering = nx.clustering(network)
clustering_values = list(clustering.values())

statistics_viz(axs[1], network, centrality_values)
statistics_viz(axs[3], network, betweenness_values)
statistics_viz(axs[5], network, clustering_values)


plt.show

### 2.3.3 Watts-Strogatz Network

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16, 10))
axs = axs.flatten()

it = 100

all_cent = np.zeros((it,1000))
all_betw = np.zeros((it,1000))
all_clus = np.zeros((it,1000))

for t in range(it):
    # Network topology: Large graph for statistics
    g_WS = nx.watts_strogatz_graph(1000, 6, 0.1)          
    network = g_WS

    # Centrality
    centrality = nx.degree_centrality(network)
    centrality_values = list(centrality.values())

    # Betweenness
    betweenness = nx.betweenness_centrality(network)
    betweenness_values = list(betweenness.values())

    # clustering
    clustering = nx.clustering(network)
    clustering_values = list(clustering.values())

    all_cent[t] = centrality_values
    all_betw[t] = betweenness_values
    all_clus[t] = clustering_values

centrality_values = [sum(group) / len(group) for group in zip(*all_cent)]
betweenness_values = [sum(group) / len(group) for group in zip(*all_betw)]
clustering_values = [sum(group) / len(group) for group in zip(*all_clus)]

statistics_plot(axs[0], network, centrality_values)
statistics_plot(axs[2], network, betweenness_values)
statistics_plot(axs[4], network, clustering_values)


# Network topology: small graph for visualization
g_WS = nx.watts_strogatz_graph(150, 6, 0.1)          
network = g_WS

# Centrality
centrality = nx.degree_centrality(network)
centrality_values = list(centrality.values())

# Betweenness
betweenness = nx.betweenness_centrality(network)
betweenness_values = list(betweenness.values())

# clustering
clustering = nx.clustering(network)
clustering_values = list(clustering.values())

statistics_viz(axs[1], network, centrality_values)
statistics_viz(axs[3], network, betweenness_values)
statistics_viz(axs[5], network, clustering_values)


plt.show

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16, 10))
axs = axs.flatten()

it = 100

all_cent = np.zeros((it,1000))
all_betw = np.zeros((it,1000))
all_clus = np.zeros((it,1000))

for t in range(it):
    # Network topology: Large graph for statistics
    g_WS = nx.watts_strogatz_graph(1000, 3, 0.05)          
    network = g_WS

    # Centrality
    centrality = nx.degree_centrality(network)
    centrality_values = list(centrality.values())

    # Betweenness
    betweenness = nx.betweenness_centrality(network)
    betweenness_values = list(betweenness.values())

    # clustering
    clustering = nx.clustering(network)
    clustering_values = list(clustering.values())

    all_cent[t] = centrality_values
    all_betw[t] = betweenness_values
    all_clus[t] = clustering_values

centrality_values = [sum(group) / len(group) for group in zip(*all_cent)]
betweenness_values = [sum(group) / len(group) for group in zip(*all_betw)]
clustering_values = [sum(group) / len(group) for group in zip(*all_clus)]

statistics_plot(axs[0], network, centrality_values)
statistics_plot(axs[2], network, betweenness_values)
statistics_plot(axs[4], network, clustering_values)


# Network topology: small graph for visualization
g_WS = nx.watts_strogatz_graph(150, 3, 0.05)          
network = g_WS

# Centrality
centrality = nx.degree_centrality(network)
centrality_values = list(centrality.values())

# Betweenness
betweenness = nx.betweenness_centrality(network)
betweenness_values = list(betweenness.values())

# clustering
clustering = nx.clustering(network)
clustering_values = list(clustering.values())

statistics_viz(axs[1], network, centrality_values)
statistics_viz(axs[3], network, betweenness_values)
statistics_viz(axs[5], network, clustering_values)


plt.show

### 2.4 Simulate SIR Disease Spread on Network

In [ ]:
def SIR_network(graph, beta, gamma, fraction_infected, model_it):
    ''' Run the SIR model on a network '''
    
    # Model selection
    model = ep.SIRModel(graph)
    
    # Model Configuration
    cfg = mc.Configuration()
    cfg.add_model_parameter('beta', beta)
    cfg.add_model_parameter('gamma', gamma)
    cfg.add_model_parameter("fraction_infected", fraction_infected)
    model.set_initial_status(cfg)
    
    # Simulation execution
    iterations = model.iteration_bunch(model_it)
    
    # Extract S, I, R trends
    Susceptible = [iteration['node_count'][0] for iteration in iterations]
    Infected = [iteration['node_count'][1] for iteration in iterations]
    Recovered = [iteration['node_count'][2] for iteration in iterations]

    return Susceptible, Infected, Recovered

    
def viz_subplot(ax, time, all_susceptible, all_infected, all_recovered, avg_susceptible, avg_infected, avg_recovered, title):
    ''' Plot different trajectories for Susceptible, Infected and Recoveren
    and the average trajectorie '''

    for susceptible, infected, recovered in zip(all_susceptible, all_infected, all_recovered):
        ax.plot(time, susceptible, alpha=.2, color="blue")
        ax.plot(time, infected, alpha=.2, color="red")
        ax.plot(time, recovered, alpha=.2, color="green")
    
    # Plot the averages
    ax.plot(time, avg_susceptible, color="blue", label="Susceptible", linewidth=2)
    ax.plot(time, avg_infected, color="red", label="Infected", linewidth=2)
    ax.plot(time, avg_recovered, color="green", label="Recovered", linewidth=2)

    # Add legend
    ax.legend(loc='upper right')
    ax.set_title(title)
    

def SIR_network_plot(time, network, betas, gammas, fraction_infected, model_it, n_it, N):
    ''' Compute the data for different SIR trajectories on a network for different parameter values,
    combining the subplots one figure '''

    dbeta = len(betas)
    dgamma = len(gammas)
    fig, axs = plt.subplots(dbeta, dgamma, figsize=(5*dbeta, 4*dgamma))
    axs = axs.flatten()
     
    j=0
    for beta in betas:
        for gamma in gammas:
            n_time_points = len(time)  

            # Create arrays
            all_S = np.zeros((n_it, n_time_points))
            all_I = np.zeros((n_it, n_time_points))
            all_R = np.zeros((n_it, n_time_points))

            for i in range(n_it):
                S, I, R = SIR_network(network, beta, gamma, fraction_infected, model_it)
                
                S = np.array(S)
                I = np.array(I)
                R = np.array(R)
                

                all_S[i] = S / N
                all_I[i] = I / N
                all_R[i] = R / N
            
            # Compute averages over all iterations
            avg_S = np.mean(all_S, axis=0)
            avg_I = np.mean(all_I, axis=0)
            avg_R = np.mean(all_R, axis=0)
                
            title = f"Beta: {beta}, Gamma: {gamma}"
            viz_subplot(axs[j], time, all_S, all_I, all_R, avg_S, avg_I, avg_R, title)
            j+=1
    # Create dummy data for each category
    #plt.plot([], [], 'ro', label='Susceptible')
    #plt.plot([], [], 'go', label='Infected')
    #plt.plot([], [], 'bo', label='Recovered')

    fig.suptitle(network_name(network), fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 1])
    save_fig(network_name(network))
    plt.show()


### 2.4.1 Erdos-Reyni Graph

In [ ]:
fraction_infected = 0.1
betas = [.1, .01]
gammas = [.01, .001]
N = 1000

model_it = 300 #time
n_it = 50 #number of plots

time = list(range(model_it))

# Network topology: Large graph for statistics
g_ER = nx.erdos_renyi_graph(N, 0.1)          
network = g_ER

SIR_network_plot(time, network, betas, gammas, fraction_infected, model_it, n_it, N)

### 2.4.2 Barabasi-Albert graph

In [ ]:
fraction_infected = 0.1
betas = [.1, .01]
gammas = [.01, .001]
N = 1000

model_it = 300 #time
n_it = 50 #number of plots

time = list(range(model_it))

# Network topology: Large graph for statistics
g_BA = nx.barabasi_albert_graph(N, 6)          
network = g_BA

SIR_network_plot(time, network, betas, gammas, fraction_infected, model_it, n_it, N)

### 2.4.3 Watts-Strogatz graph

In [ ]:
fraction_infected = 0.1
betas = [.1, .01]
gammas = [.01, .001]
N = 1000

model_it = 300 #time
n_it = 50 #number of plots

time = list(range(model_it))

# Network topology: Large graph for statistics
g_WS = nx.watts_strogatz_graph(N, 6, 0.1)          
network = g_WS

SIR_network_plot(time, network, betas, gammas, fraction_infected, model_it, n_it, N)

In [ ]:
fraction_infected = 0.1
betas = [.1, .01]
gammas = [.01, .001]
N = 1000

model_it = 300 #time
n_it = 50 #number of plots

time = list(range(model_it))

# Network topology: Large graph for statistics
g_WS = nx.watts_strogatz_graph(N, 3, 0.05)          
network = g_WS

SIR_network_plot(time, network, betas, gammas, fraction_infected, model_it, n_it, N)